In [8]:
import mlflow
import mlflow.sklearn
import os
import platform
import psutil
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score, recall_score,f1_score
from mlflow.tracking import MlflowClient


In [9]:
# Global Parameters
def get_config():
    return {
        "learning_rate": 0.01,
        "batch_size":32,
        "n_estimators": 10,
        "max_depth":3
    }

In [10]:
#Define training function
def train_model(config):
    data = load_iris()
    X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(n_estimators=config["n_estimators"], max_depth=config["max_depth"], random_state=42)
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='macro') # Weighted for multi-class  
    recall = recall_score(y_test, predictions, average='weighted')  # Weighted for multi-class
    f1= f1_score(y_test, predictions, average='weighted')
    
    return model, accuracy, predictions, y_test, precision, recall, f1

In [11]:

# Define logging function
def log_model_mlflow(config, model, accuracy, predictions, y_test, precision, recall, f1):
    mlflow.set_tracking_uri(uri='http://127.0.0.1:5000/')
    mlflow.set_experiment('iris Classifier Insights')
    
    model_name = "random forest, iris"

    with mlflow.start_run():
        # Log parameters
        for param_name, param_value in config.items():
            mlflow.log_param(param_name, param_value)

        # Log metrics
        mlflow.log_metric('accuracy', accuracy)
        mlflow.log_metric('precision', precision)
        mlflow.log_metric('recall', recall)
        mlflow.log_metric('f1', f1)
        loss = np.mean(np.square(y_test - predictions))  # Mean Squared Error
        mlflow.log_metric('loss', loss)

        # Log system information
        mlflow.log_param("system_os", platform.system())
        mlflow.log_param("system_processor", platform.processor())
        mlflow.log_param("system_ram", f"{psutil.virtual_memory().total / 1e9:.2f} GB")
        mlflow.log_param("cpu_count", os.cpu_count())
        mlflow.log_param("python_version", platform.python_version())

        # Log detailed hardware information
        cpu_freq = psutil.cpu_freq().max if psutil.cpu_freq() else "Unknown"
        disk_usage = psutil.disk_usage('/').total / 1e9  # Disk size in GB
        mlflow.log_param("cpu_max_frequency_MHz", cpu_freq)
        mlflow.log_param("disk_total_GB", f"{disk_usage:.2f}")

        # Measure resource usage
        start_time = time.time()
        initial_memory = psutil.Process(os.getpid()).memory_info().rss / 1e6  # Memory in MB

        runtime = time.time() - start_time
        final_memory = psutil.Process(os.getpid()).memory_info().rss / 1e6  # Memory in MB
        memory_used = final_memory - initial_memory

        mlflow.log_metric("runtime", runtime)
        mlflow.log_metric("memory_used_MB", memory_used)

        # Log model artifact
        mlflow.sklearn.log_model(model, "random forest, iris")
      
        
        # Task A: Register a model ----> "random forest, iris"
        model_uri = f"runs:/{mlflow.active_run().info.run_id}/model"
        print("model_uri:", model_uri)
        
        try:
            registered_model = mlflow.register_model(model_uri=model_uri, name=model_name)
            print(f"Model '{model_name}' registered successfully as version {registered_model.version}.")
        except Exception as e:
            print(f"Error during model registration: {e}")
            return
        
        # Task B: Transition model stages 
        client = MlflowClient()
        try:
            # Transition to "Staging"
            client.transition_model_version_stage(
                name=model_name,
                version=registered_model.version,
                stage="Staging"
            )
            print(f"Model '{model_name}', version {registered_model.version} transitioned to 'Staging' stage.")
            
            # Check predefined criteria (e.g., accuracy > threshold)
            ACCURACY_THRESHOLD = 0.90
            if accuracy > ACCURACY_THRESHOLD:
            # Register new version for Production (version 2)
                model_uri = f"runs:/{mlflow.active_run().info.run_id}/model"
                registered_model_production = mlflow.register_model(model_uri=model_uri, name=model_name)
                print(f"New model version {registered_model_production.version} registered for 'Production'.")

                # Transition from Staging to Production
                client.transition_model_version_stage(
                    name=model_name,
                    version=registered_model_production.version,
                    stage="Production"
            )
                
                print(f"Model '{model_name}', version {registered_model.version} transitioned to 'Production' stage because accuracy is greater than {ACCURACY_THRESHOLD}.")
            else:
                print(f"Model '{model_name}', version {registered_model.version} did not meet accuracy threshold and remains in 'Staging'.")
        
        except Exception as e:
            print(f"Error during stage transition: {e}")
       

In [12]:
# Main Execution
    
if __name__ == "__main__":
    try:
        # Get the configuration for the model
        config = get_config()
       # Train the model and log the necessary variables
        model, accuracy, predictions, y_test, precision, recall, f1 = train_model(config)

        # Log the model to MLflow
        log_model_mlflow(config, model, accuracy, predictions, y_test, precision, recall, f1)
    except Exception as e:
        print(f"An error occurred during the main execution: {e}")    
  

2024/12/21 02:12:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


model_uri: runs:/59374e9336cb46e8b1677e2f0bb68caf/model


Successfully registered model 'random forest, iris'.
2024/12/21 02:12:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random forest, iris, version 1
Created version '1' of model 'random forest, iris'.
C:\Users\Zahra\AppData\Local\Temp\ipykernel_10364\2196517583.py:64: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Model 'random forest, iris' registered successfully as version 1.
Model 'random forest, iris', version 1 transitioned to 'Staging' stage.


Registered model 'random forest, iris' already exists. Creating a new version of this model...
2024/12/21 02:12:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random forest, iris, version 2
Created version '2' of model 'random forest, iris'.
C:\Users\Zahra\AppData\Local\Temp\ipykernel_10364\2196517583.py:80: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


New model version 2 registered for 'Production'.
Model 'random forest, iris', version 1 transitioned to 'Production' stage because accuracy is greater than 0.9.
🏃 View run thoughtful-rook-838 at: http://127.0.0.1:5000/#/experiments/135740183885560034/runs/59374e9336cb46e8b1677e2f0bb68caf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/135740183885560034


In [13]:

# function to compare different model versions
def compare_model_versions(model_name):
    """
    Compares different versions of a model registered in the MLflow Model Registry.
    """
    client = MlflowClient()
    try:
        # Fetch all registered versions of the model
        versions = client.get_latest_versions(name=model_name)
        print(f"Model '{model_name}' has the following versions:")
        for version in versions:
            print(f"Version: {version.version}, Stage: {version.current_stage}, Run ID: {version.run_id}")
            
            # Fetch metrics and params for each version
            run = client.get_run(version.run_id)
            metrics = run.data.metrics
            params = run.data.params
            print(f"Metrics: {metrics}")
            print(f"Parameters: {params}")
            print("-" * 50)
    except Exception as e:
        print(f"Error while comparing model versions: {e}")

In [14]:

model_name = "random forest, iris"  # Set the name of the model
compare_model_versions(model_name)   # Compare different versions of the model

C:\Users\Zahra\AppData\Local\Temp\ipykernel_10364\570866499.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(name=model_name)


Model 'random forest, iris' has the following versions:
Version: 2, Stage: Production, Run ID: 59374e9336cb46e8b1677e2f0bb68caf
Metrics: {'accuracy': 1.0, 'f1': 1.0, 'loss': 0.0, 'memory_used_MB': 0.012287999999998078, 'precision': 1.0, 'recall': 1.0, 'runtime': 0.07064414024353027}
Parameters: {'batch_size': '32', 'cpu_count': '4', 'cpu_max_frequency_MHz': '2501.0', 'disk_total_GB': '157.29', 'learning_rate': '0.01', 'max_depth': '3', 'n_estimators': '10', 'python_version': '3.11.9', 'system_os': 'Windows', 'system_processor': 'Intel64 Family 6 Model 58 Stepping 9, GenuineIntel', 'system_ram': '4.18 GB'}
--------------------------------------------------
Version: 1, Stage: Staging, Run ID: 59374e9336cb46e8b1677e2f0bb68caf
Metrics: {'accuracy': 1.0, 'f1': 1.0, 'loss': 0.0, 'memory_used_MB': 0.012287999999998078, 'precision': 1.0, 'recall': 1.0, 'runtime': 0.07064414024353027}
Parameters: {'batch_size': '32', 'cpu_count': '4', 'cpu_max_frequency_MHz': '2501.0', 'disk_total_GB': '157.29'

In [6]:
#optional

# Function to delete an entire registered model and all its versions
def delete_registered_model(model_name):
    try:
        client = MlflowClient()
        print(f"Deleting registered model '{model_name}' and all its versions...")
        
        # Delete all versions of the model
        versions = client.get_latest_versions(name=model_name)
        for version in versions:
            client.delete_model_version(name=model_name, version=version.version)
        
        # Delete the registered model
        client.delete_registered_model(name=model_name)
        print(f"Registered model '{model_name}' deleted successfully.")
    except Exception as e:
        print(f"An error occurred while deleting registered model '{model_name}': {e}")


In [7]:
#optional
# Completely delete the model and all its versions
model_name = "random forest, iris"   # Set the name of the model
delete_registered_model(model_name) # Delete the registered model and all versions

Deleting registered model 'random forest, iris' and all its versions...


C:\Users\Zahra\AppData\Local\Temp\ipykernel_10364\3968451189.py:10: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(name=model_name)


Registered model 'random forest, iris' deleted successfully.


In [25]:
# optional

#Delete specific version of a model

# Function to delete a specific version of a model
def delete_model_version(model_name, version):
    try:
        client = MlflowClient()
        print(f"Deleting version {version} of model '{model_name}'...")
        client.delete_model_version(name=model_name, version=version)
        print(f"Version {version} of model '{model_name}' deleted successfully.")
    except Exception as e:
        print(f"An error occurred while deleting version {version} of model '{model_name}': {e}")

In [27]:
#optional
# Delete a specific version of a model
model_name = "random forest, iris"  # Set the name of the model
version_to_delete = 2  # The version of the model you want to delete
delete_model_version(model_name, version_to_delete)

Deleting version 2 of model 'random forest, iris'...
Version 2 of model 'random forest, iris' deleted successfully.
